In [132]:
import torch
import numpy as np
import random
import opensmile

import pickle

import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import os

In [133]:
torch.__version__

'1.13.1+cu117'

In [134]:
rootdir = '../data_2'
data_list = []
with open(f'../data_base_2.pickle', 'rb') as fh:
  data_list = pickle.load(fh)

In [135]:
data_list[0]

{'name': '29b4d0958882b4866ca723a3d1548e31_SNR_3',
 'marks': array([0, 0, 0, ..., 0, 0, 0], dtype=int8)}

In [136]:
path_data = f'{rootdir}/'

end_idx_tv = 87000
train_valid_subset = data_list[:end_idx_tv]
test_subset = data_list[end_idx_tv:]

lengths = [int(len(train_valid_subset) * 0.9), int(len(train_valid_subset) * 0.1)]
train_subset, valid_subset = torch.utils.data.random_split(train_valid_subset, lengths)
len(train_subset)

78300

In [137]:
random.seed(123456)
np.random.seed(123456)
torch.manual_seed(123456)

In [138]:
smile = opensmile.Smile(feature_set=opensmile.FeatureSet.emobase,
                feature_level=opensmile.FeatureLevel.LowLevelDescriptors)
smile.process_file(f'{path_data}/5f4edcae839e9c7800bd2781ff6cfa71_SNR_3.wav')

pcm_intensity_sma  \
file                                               start                  end                                         
../data_2//5f4edcae839e9c7800bd2781ff6cfa71_SNR... 0 days 00:00:00        0 days 00:00:00.025000       1.483837e-07   
                                                   0 days 00:00:00.010000 0 days 00:00:00.035000       2.447667e-07   
                                                   0 days 00:00:00.020000 0 days 00:00:00.045000       2.668529e-07   
                                                   0 days 00:00:00.030000 0 days 00:00:00.055000       4.475613e-07   
                                                   0 days 00:00:00.040000 0 days 00:00:00.065000       6.832497e-07   
...                                                                                                             ...   
                                                   0 days 00:00:05.930000 0 days 00:00:05.955000       2.436783e-07   
                                                   0 days 00:00:05.940000 0 days 00:00:05.965000       1.800450e-07   
                                                   0 days 00:00:05.950000 0 days 00:00:05.975000       1.753107e-07   
                                                   0 days 00:00:05.960000 0 days 00:00:05.985000       1.553826e-07   
                                                   0 days 00:00:05.970000 0 days 00:00:06              1.224016e-07   

                                                                                                  pcm_loudness_sma  \
file                                               start                  end                                        
../data_2//5f4edcae839e9c7800bd2781ff6cfa71_SNR... 0 days 00:00:00        0 days 00:00:00.025000          0.563944   
                                                   0 days 00:00:00.010000 0 days 00:00:00.035000          0.637391   
                                                   0 days 00:00:00.020000 0 days 00:00:00.045000          0.660013   
                                                   0 days 00:00:00.030000 0 days 00:00:00.055000          0.767033   
                                                   0 days 00:00:00.040000 0 days 00:00:00.065000          0.854613   
...                                                                                                            ...   
                                                   0 days 00:00:05.930000 0 days 00:00:05.955000          0.642737   
                                                   0 days 00:00:05.940000 0 days 00:00:05.965000          0.597341   
                                                   0 days 00:00:05.950000 0 days 00:00:05.975000          0.592329   
                                                   0 days 00:00:05.960000 0 days 00:00:05.985000          0.567732   
                                                   0 days 00:00:05.970000 0 days 00:00:06                 0.530298   

                                                                                                  mfcc_sma[1]  \
file                                               start                  end                                   
../data_2//5f4edcae839e9c7800bd2781ff6cfa71_SNR... 0 days 00:00:00        0 days 00:00:00.025000   -31.718285   
                                                   0 days 00:00:00.010000 0 days 00:00:00.035000   -31.582565   
                                                   0 days 00:00:00.020000 0 days 00:00:00.045000   -30.977066   
                                                   0 days 00:00:00.030000 0 days 00:00:00.055000   -31.748747   
                                                   0 days 00:00:00.040000 0 days 00:00:00.065000   -31.736212   
...                                                                                                       ...   
                                                   0 days 00:00:05.930000 0 days 00:00:05.955000   -32.968582  

In [139]:
def create_marks(old_marks):
    data_mark = []
    step = 81
    for window in np.arange(0, len(old_marks)-step, step):
        values, counts = np.unique(old_marks[window:step+window], return_counts=True)
        max_count = max(counts)
        most_frequent = values[counts == max_count]
        data_mark.append(most_frequent[0])
    return data_mark

def db_create(subset, common_db):
    xname = subset['name']
    temp = smile.process_file(f"{rootdir}/{xname}.wav")
    temp = temp.iloc[:592]
    old_marks = subset['marks']    
    temp['marks'] = create_marks(old_marks)
    common_db = pd.concat([common_db, temp], axis=0)
    return common_db

In [140]:
train_subset[0]['marks']

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [ ]:
common_db = pd.DataFrame()
for i in tqdm(range(10000)):
    common_db = db_create(train_subset[i], common_db)

common_db.to_csv('common_db.csv', sep='\t')
common_db

  0%|          | 0/10000 [00:00<?, ?it/s]

In [173]:
len(common_db['marks'])

5920000

In [174]:
feature_db = pd.read_csv('common_db.csv', sep='\t')
feature_db

,file,start,end,pcm_intensity_sma,pcm_loudness_sma,mfcc_sma[1],mfcc_sma[2],mfcc_sma[3],mfcc_sma[4],mfcc_sma[5],...,lspFreq_sma[3],lspFreq_sma[4],lspFreq_sma[5],lspFreq_sma[6],lspFreq_sma[7],pcm_zcr_sma,voiceProb_sma,F0_sma,F0env_sma,marks
0,../data_2/7a263d8418b34994ad76a38298347100_SNR...,0 days 00:00:00,0 days 00:00:00.025000,1.437355e-06,1.087248,-28.810083,-14.408603,-13.337047,-3.625848,-5.591749,...,1.517261,1.850826,2.174934,2.550767,2.779072,0.446667,0.167102,0.0,0.00000,0
1,../data_2/7a263d8418b34994ad76a38298347100_SNR...,0 days 00:00:00.010000,0 days 00:00:00.035000,1.391134e-06,1.078037,-29.527891,-13.265147,-11.936412,-5.146031,-4.833334,...,1.537388,1.859312,2.175371,2.525171,2.787575,0.458333,0.174306,0.0,0.00000,0
2,../data_2/7a263d8418b34994ad76a38298347100_SNR...,0 days 00:00:00.020000,0 days 00:00:00.045000,8.546493e-07,0.897381,-31.317423,-11.796272,-12.508189,-6.910802,-3.441063,...,1.554669,1.873642,2.173566,2.503890,2.801077,0.473333,0.191142,0.0,0.00000,0
3,../data_2/7a263d8418b34994ad76a38298347100_SNR...,0 days 00:00:00.030000,0 days 00:00:00.055000,1.004138e-06,0.950276,-32.413956,-10.902612,-11.026330,-7.137033,-5.975752,...,1.562508,1.893190,2.187434,2.503631,2.814847,0.495000,0.199972,0.0,0.00000,0
4,../data_2/7a263d8418b34994ad76a38298347100_SNR...,0 days 00:00:00.040000,0 days 00:00:00.065000,9.871719e-07,0.946765,-32.620213,-9.060246,-12.826831,-6.863073,-8.375133,...,1.560471,1.900912,2.186834,2.526136,2.824278,0.496667,0.211909,0.0,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5919995,../data_2/3aa611104e76162423c8002902ed0eda_SNR...,0 days 00:00:05.870000,0 days 00:00:05.895000,1.088412e-05,2.015965,-34.167480,-11.865521,-9.812234,-7.619404,-7.540821,...,1.564088,1.874483,2.176894,2.513136,2.836721,0.508333,0.177433,0.0,222.92432,0
5919996,../data_2/3aa611104e76162423c8002902ed0eda_SNR...,0 days 00:00:05.880000,0 days 00:00:05.905000,1.126486e-05,2.047166,-34.481228,-12.976173,-12.759544,-8.113576,-9.710672,...,1.563800,1.895140,2.162098,2.536663,2.827542,0.528333,0.170412,0.0,222.92432,0
5919997,../data_2/3aa611104e76162423c8002902ed0eda_SNR...,0 days 00:00:05.890000,0 days 00:00:05.915000,7.954719e-06,1.748897,-34.321020,-12.302844,-13.334003,-9.151077,-6.535006,...,1.559156,1.898577,2.149552,2.556510,2.828760,0.530000,0.179985,0.0,222.92432,0
5919998,../data_2/3aa611104e76162423c8002902ed0eda_SNR...,0 days 00:00:05.900000,0 days 00:00:05.925000,3.958584e-06,1.463524,-32.623566,-11.021976,-15.553813,-10.456256,-6.526885,...,1.569478,1.898470,2.145011,2.560770,2.817395,0.520000,0.157476,0.0,222.92432,0


In [175]:
values = list(feature_db.columns)[3:]

In [176]:
values

['pcm_intensity_sma',
 'pcm_loudness_sma',
 'mfcc_sma[1]',
 'mfcc_sma[2]',
 'mfcc_sma[3]',
 'mfcc_sma[4]',
 'mfcc_sma[5]',
 'mfcc_sma[6]',
 'mfcc_sma[7]',
 'mfcc_sma[8]',
 'mfcc_sma[9]',
 'mfcc_sma[10]',
 'mfcc_sma[11]',
 'mfcc_sma[12]',
 'lspFreq_sma[0]',
 'lspFreq_sma[1]',
 'lspFreq_sma[2]',
 'lspFreq_sma[3]',
 'lspFreq_sma[4]',
 'lspFreq_sma[5]',
 'lspFreq_sma[6]',
 'lspFreq_sma[7]',
 'pcm_zcr_sma',
 'voiceProb_sma',
 'F0_sma',
 'F0env_sma',
 'marks']

mkdir: cannot create directory ‘dist_emobase’: File exists


In [1]:
for value in values[:1]:
    dist_graph = sns.boxplot(data=feature_db, x="marks", y=value)
    fig = dist_graph.get_figure()
    fig.suptitle(f'{value}', fontsize=20)
    fig.savefig(f"dist_emobase/{value}.png") 
    plt.show()


NameError: name 'values' is not defined